# CHAPTER.9 UP and Running with Tensorflow

* _TensorFlow_ 는 수치 계산을 위한 강력한 오픈 소스 소프트 웨어 라이브러리


* 기본 원칙 : python에서 계산을 수행 할 그래프 정의 -> 텐서플로어가 그래프를 가져와 최적화 된 c++ 코드를 사용해 효율적으로 실행


* 가장 중요한 점은 그림 9-2와 처럼 그래프를 여러개의 덩어리로 나눠, 여러개의 CPU, GPU에서 병렬로 수행 

    텐서플로어는 분산 컴퓨팅을 지원 -> 계산을 수백 대의 서버로 나눠 많은 양의 학습 데이터를 훈련시킬 수 있음


* 수십억개의 인스턴스와 수백만개의 피쳐로 구성된 학습 세트에서 수백만 개의 파라미터가 있는 네트워크를 학습시킬 수 있음

텐서플로우의 설계, 확장성, 유연성을 통해 가장 많이 사용

<주요 특징>

* 윈도우, 리눅스, 맥OS 뿐 아니라, iOS, Android 에도 실행 가능


* TF.Learn2라는 매우 간단한 파이썬 API 제공
    -> 몇 줄의 코드만으로 다양한 유형의 신경망 훈련
    
    
* TF-slim이라는 간단한 API를 제공 -> 신경망을 빌드, 학습, 평가하는 작업을 단순화


* Keras or Pretty Tensor와 같이 텐서플로우 위에 여러가지 고수준의 API를 독립적으로 구축


* 주요 파이썬 API는 신경망 구조를 포함해 모든 종류의 계산을 훨씬 유연하게 제공


* 특히 뉴럴 네트워크를 빌드하기 위해 필요한 많은 ML 작업의 고효율적인 C++ 구현이 포함


- 비용함수를 최소화하는 파라미터를 검색할 수 있는 몇가지 고급 최적화 노드를 제공
   
   텐서플로우는 사용자가 정의한 함수의 그래디언트를 자동을 계산하므로 사용하기가 쉬움
 
 -> _autodiff(automatic differentiaing)_ 라고 한다.
    
- TensorBoard라는 시각화 도구가 있어 계산 그래프(computation graph)를 탐색하고 학습 곡선을 보는 작업 수행 가능


- 구글은 텐서플로우 그래프를 실행하기 위해 클라우드 서비스를 시작



이번 챕터에서는, 설치부터 간단한 계산, 그래프의 생성, 실행, 저장 및 시각화에 이르는 텐서플로우의 기본 사항을 살핌

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# 1. Installation

2장에서 Jupyter와 Scikit-Learn를 설치했다!

이제 pip로 쉽게 텐서플로우를 설치 할 수 있다.


# 2. Creating Your First Graph and Running It in a Session


아래는 그래프를 만드는 코드

In [2]:
import tensorflow as tf


x=tf.Variable(3, name='x')
y=tf.Variable(4, name='y')
f=x*x*y+y+2

/home/soyeon/sy/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


중요한 점은, 위의 코드는 실제로 계산을 수행하지 않는다.
단지 계산 그래프를 생성!!

실제로 변수도 초기화 되지 않음

-> 이 그래프를 평가하기 위해 네서플로우의 session을 열고, 변수를 초기화해, f를 평가해야함

텐서플로우의 세션은 CPU or GPU와 같은 장치에 작업을 배치하고, 실행하며 모든 변수 값을 보유

아래 코드는 세션을 만들고, 변수를 초기화하여 평가한 다음 세션을 닫는다.

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

sess.run()을 반복하는건 번거롭다.

따라서 아래 코드는 더 간단한 방법을 제공한다!

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
    
print(result)

42


__with__ 블록의 세션은 기본 세션으로 설정된다.

__x.initializer.run()__ 을 호출하는 것은 __tf.get_default_session()__ 을 호출하는 것과 동일

마찬가지로 __f.eval()__ 은 __tf.get_default_session.run(f)__ 를 호출 하는 것과 동일

이렇게 한다면 코드 읽기 쉬워짐

또한 세션이 블록의 끝에서 자동으로 닫히게 된다.


단일 변수마다 수동적으로 초기화를 하는 것 대신, __global_variables_initializer()__ 함수를 사용 할 수 있다.

실제로 초기화를 수행하지는 않지만, 그래프가 실행 될 때 모든 변수를 초기화 할 노드를 만드는 것


In [8]:
init = tf.global_variables_initializer() #init node준비

with tf.Session() as sess:
    init.run() #실제로 이 부분에서 모든 변수 초기화
    result=f.eval()

In [9]:
print(result)

42


주피터나 파이선 셀 내에서는 __InteractiveSession__ 을 만드는 것이 더 좋다.

일반 세션과 차이점은 __InteractiveSession__ 이 생성되면, 자동으로 기본 세션으로 설정되므로 with 블록은 필요하지 않다! (단, 작업이 끝난 후 수동으로 세션을 닫아야 된다!)

In [11]:
init = tf.global_variables_initializer() #init node준비

sess=tf.InteractiveSession()
init.run()
result=f.eval()
print(result)
sess.close()

42


텐서플로우 프로그램은 두가지로 나눠진다.

첫 번째 부분은 계산 그래프를 빌드 (=_construction phase_) 

두 번째 부분은 실행 (=_execution phase_)

construction phase(구축단계)에서는 일반적으로 머신러닝 모델과 이를 학습시키기 위해 필요한 계산을 나타내는 계산 그래프를 빌드

execution phase(실행단계)에서는 일반적으로 학습을 반복적으로 평가하는 루프를 실행하며, 점진적으로 모델 파라미터를 향상시킴

# 3. Managing Graphs

생성한 노드는 자동적으로 기본 그래프에 추가된다.



In [12]:
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()

True

대부분은 잘 맞지만, 때때로 여러 개의 독립적인 그래프를 관리 해야 할 수도 있다.

새로운 그래프를 만들고 일시적으로 기본 그래프를 with 블록 안에 만들어야 한다면, 아래처럼 할 수 있다.


In [13]:
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
    
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

# 4. Lifecycle of a Node Value

노드를 평가할 때, 텐서플로우는 의존적인 노드 집합을 자동으로 결정하고, 이 노드르 먼저 평가



In [15]:
w=tf.constant(3)
x=w+2
y=x+5
z=x*3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())


10
15


첫번째로, 위의 코드는 매우 간단한 그래프를 정의한다.

그런 다음, 세션을 시작하고 그래프를 실행해 y를 평가

텐서플로우는 자동으로 y는 x에 의존적이라는 것을 감지 (x는 w에 의존 포함)

따라서, 처음으로 w를 평가, x평가, y평가 그리고 y값 반환

마지막으로 코드는 그래프를 실행해, z를 평가

다시한번, 텐서플로우는 w,x를 평가함을 감지

    (w, x의 이전 평가 결과를 사용하지 않는다)

간단히 말해 위의 코드는 w와 x를 두 번 계산

만약 효율적으로 y,z를 평가하기 원한다면 아래 코드를 사용할 수 있다!

텐서플로우는 y,z를 하나의 그래프로 실행 시킬 수 있다.

In [16]:
with tf.Session() as sess:
    y_val, z_val =sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


# 5. Linear Regression with Tensorflow

텐서플로우 operations(ops라 부름)은 임의의 수의 입력값을 가져와서 여러 개의 출력값을 생성 할 수 있다.

예를 들어, 덧셈과 곱셈 연산은 각각 2개의 입력을 취해 하나의 출력을 생성

상수와 변수는 입력이 없다(_source ops_ 라 부름)

입력과 출력은 _tensor_ 라고 하는 다차원 배열

Numpy배열과 마찬가지로 텐서는 type와 shape를 가진다.

실제로, Python API에서 텐서는 단순히 numpy ndarrays로 표현된다.

일반적으로 부동 소수점을 포함하지만, 문자열을 전달하는 데에도 사용 할 수 있다.



지금까지, 텐서는 단지 하나의 스칼라 값을 포함하고 있지만, 여러 shape한 배열에 대해서도 계산을 수행 할 수 있다.

예를 들어, 아래 코드는 2차원 배열을 조작해, 캘리포니아 주택 데이터 세트에서 선형 회귀를 수행

데이터셋을 부르고, 모든 학습 인스턴스에 extra bias input feature (x0=1)을 추가

그런 다음, 두 개의 텐서플로우 상수 노드인 X와 y를 만들어서 데이터와 타겟을 넣어주고, 텐서플로우에서 제공하는 일부 행렬 계산을 사용해 theta를 정의

행렬 함수는 _(__transepose(), matmul(), maxtrix_inverse()__ 를 사용)_ 는 자명하지만, 평소와 같이 즉시 계산을 수행하지 않는다.

대신에, 그래프가 실행될때, 수행하는 그래프안에 노드를 생성한다.

theta의 정의 정규 방정식과 일치하는 것을 알 수 있다.

마지막으로 코드는 세션을 만들고, 이를 사용해 theta를 평가


In [17]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()

m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)), housing.data]

X=tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value=theta.eval()


In [18]:
theta_value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

# 6. Implementing Gradient Descent

정규 방정식 대신 Batch Gradient Descent를 사용해보자!!

먼저 그래디언트를 수동으로 계산한다.

다음으로 텐서플로우의 autodiff기능을 사용해 텐서플로우가 자동으로 그래디언트를 계산하도록 할 것이다.

마지막으로 텐서플로우의 기본 optimizer을 이용 할 것!



            
           * 주의

        Gradient Descent를 이용할 때, 우선적으로 입력 피쳐 벡터에 대해 정규화(normalize)하는 것이 중요

## 6.1 Manually Computing the Gradients

다음 코드는 몇가지 새로운 요소를 제외하고는 자명하다.

* __random_uniform()__ 함수는 numpy rand() 함수와 마찬가지로 주어진 shape와 value의 범위 안에서 무작위 값을 포함하는 테서를 생성하는 그래프의 노드를 생성.


* __assign()__ 함수는 새로운 값을 변수에 할당 할 노드를 생성


* main loop는 학습 단계를 반복 실행하고, 매 100회 반복 할 때마다, 현재 Mean Squared Error(__mse__)를 출력

        매 반복마다 mse가 내려가는 것을 확인 할 수 있다.
        


In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [21]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [22]:

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.53382194
Epoch 800 MSE = 0.53124255
Epoch 900 MSE = 0.5293705


In [23]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192401e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081335e-04],
       [-3.91945131e-02],
       [-8.61356676e-01],
       [-8.23479772e-01]], dtype=float32)

## 6.2 Using autodiff

위 코드는 정상적으로 작동하지만, 비용함수에서 그래디언트를 수학적으로 유도해야한다.

선형 회귀 분석의 경우에는 쉽지만, 신경망 네트워크를 사용할 때는 매우 어렵고 오류 발생하기 쉬움

기호 미분(_symbolic differentiation_)을 사용해 편도함수의 방정식을 자동으로 찾을 수 있지만,  결과 코드가 반드시 효율적이지 않음

텐서플로우에 autodiff 기능은 자동으로 효율적인 그래디언트를 계산할 수 있다.


    * gradients= ... 행 부분을 gradient=tf.grdients(mse,[theta])[0] 로 바꾼다

In [24]:

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [25]:
gradients = tf.gradients(mse, [theta])[0]

In [26]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.6322219
Epoch 200 MSE = 0.5727803
Epoch 300 MSE = 0.5585008
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373791
Epoch 700 MSE = 0.53382194
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705


In [27]:


print("Best theta:")
print(best_theta)

Best theta:
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44077959e-04]
 [-3.91945131e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


gradients() 함수는 op와 변수 목록을 취하고, 각 op에 대한 op의 gradient를 계산하기 위해 하나의 변수당 하나의 ops 목록을 만든다.

그래디언트 노드는 theta에 관한 MSE의 그래디언트 벡터를 계산

텐서플로우는 _reverse-mode autodiff_ 를 사용

이 모드는 신경망에 흔히 있는 것처럼 많은 입력과 출력이 있을 때, 효율적이며 정확함

## 6.3 Using an Optimizer

텐서플로우는 그래디언트를 계산

gradient descent optimizer를 포함해 많은 최적화 도구를 제공

위의 코드에서 gradients= ... 와 training_op=....를

    * -> gradient=tf.gradients(mse,[theha])[0]
         optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
         
         training_op=optimizer.minimize(mse)
         
         
         로 바꾸면 된다!
         

### Using a `GradientDescentOptimizer`

In [28]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [29]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [30]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [32]:
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


만약 다른 유형의 optimizer를 사용하려면 한줄만 바꾸면 된다!

예를들어 momentum optimizer를 이용한다고 했을때, 다음과 같다.

optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

### Using a `momentum optimizer`


In [33]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [34]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [35]:
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [36]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        sess.run(training_op)
    
    best_theta = theta.eval()

In [38]:
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.068558  ]
 [ 0.82962847]
 [ 0.11875335]
 [-0.26554456]
 [ 0.3057109 ]
 [-0.00450249]
 [-0.03932662]
 [-0.8998645 ]
 [-0.8705207 ]]


# 7. Feeding Data to the Training Algorithm

이전 코드를 수정하여 mini-batch gradient descent를 구현해보자

이를 위해서 매 반복마다 X와 y를 다음 mini-batch로 교체 할 방법이 필요

-> 가장 간단한 방법은 placeholder 노드를 이용

이 노드는 실제로 계산을 수행하지 않기 때문에 특별, 런타임시 출력하도록 지정한 데이터만 출력

이들은 일반적으로 학습 중에 텐서플로우에 학습 데이터를 전달하는 데 사용

런타임 시 placeholder에 값을 지정하지 않으면, exception이 발생

placeholder 노드를 만들기 위해, __placeholder()__ 함수를 호출하고 출력 텐서의 데이터 타입을 지정

또한 shape를 지정 가능

차원에 __None__ 을 지정하면 'any size'의미

예를 들어, 아래 코드는 placeholder 노드 A와, 노드 B=A+5 를 만든다.

B를 평가할때, A값을 명시하는 __eval()__ 메소드에 __feed_dict__ 를 전달

A는 랭크 2(2차원)이며, 세개의 columns를 가짐 - 행의 개수 제한 X



In [39]:

A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[6. 7. 8.]]


In [40]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


mini-batch gradient descent를 구현하려면 

construction phase에서 X와 y의 정의를 변경해 placeholder 노드를 만듦

In [41]:
n_epochs = 1000
learning_rate = 0.01

In [44]:
reset_graph()

#X,y placeholder 노드를 만듦

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [45]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [46]:
n_epochs = 10

In [49]:
# batch size와 batches의 전체 개수를 계산

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [50]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

In [51]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255152],
       [-0.31211877],
       [ 0.38510376],
       [ 0.00434168],
       [-0.0123295 ],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

# 8. Saving and Restoring Models

모델을 학습 한 후에는, 파라미터를 디스크에 저장해야 원할 때 언제든지 다시 불러와 다른 프로그램에서 사용하고 다른 모델과 비교하는 등의 작업을 수행할 수 있다.

또한 학습 도중 일정한 간격으로 체크포인트를 저장하여 컴퓨터가 다운됐을 때 처음부터 다시 시작하지 않고 마지막 체크포인트부터 계속 할 수 있다.

텐서플로우를 사용하면, 모델을 쉽게 저장하고 복원 할 수 있다!!

모든 변수 노드가 생성 된 후 construction phase 마지막 부분에 __Saver__ 노드를 만듦

그 다음, execution phase에서 체크 포인트 파일의 세션과 경로를 전달해 모델을 저장 할 때마다, __save()__ 메소드를 호출하면 됨

In [52]:
reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver() #saver 노드 생성

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [54]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.3477088 ],
       [ 0.36178365],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

모델을 복원하는 것 또한 쉽다!!

__Saver__ 를 construction phase 끝에 만들고, execution phase의 시작 부분에 __init__ 노드를 이용해 변수를 초기화 하는 대신, __Saver__ 객체의 __restore()__ 메소드를 호출하면 된다.

In [55]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [56]:
np.allclose(best_theta, best_theta_restored)

True

In [57]:
print(best_theta_restored)

[[ 2.0685523 ]
 [ 0.8874027 ]
 [ 0.14401656]
 [-0.3477088 ]
 [ 0.36178365]
 [ 0.00393811]
 [-0.04269556]
 [-0.66145283]
 [-0.6375278 ]]


기본적으로 __Saver__ 는 모든 변수를 자체 이름으로 저장 및 복원하지만, 더 많은 제어가 필요한 경우, 저장/복원 할 변수와 사용할 이름을 지정 할 수 있다.


예를 들어, 아래 코드는 __Saver__ 는 `weight` 이름 아래 `theta` 변수만을 저장하거나 복원


In [58]:
saver = tf.train.Saver({"weights": theta})

기본적으로, __save()__ 메소드는 동일한 이름의 __.meta__ 확장자를 가진 두 번째 파일에 그래프 구조를 저장

`tf.train.import_meta_graph()` 를 사용해 이 그래프 구조를 로드할 수 있다.

-> 이렇게 하면, 기본 그래프에 그래프가 추가되고, saver 인스턴스가 반환되어 그래프의 상태를 복원하는 데 사용할 수 있다.

In [59]:
reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [60]:
np.allclose(best_theta, best_theta_restored)

True

# 9. Visualizing the Graph and Training Curves Using TensorBoard

선형 회귀를 mini-batch gradient descent를 이용해 학습하는 계산 그래프를 얻었으며, 주기적인 간격으로 체크포인트를 저장했다.

하지만 아직 `print()` 함수를 사용해 시각화 하고있다.

더 좋은 방법인 `TensolBoard`를 이용해보자!


학습 통계를 feed하면 이것을 웹 브라우저에 시각적으로 표시 할 수 있다.

또한 그래프의 정의를 제공할 수 있고, 이를 통해 탐색 할 수 있는 좋은 인터페이스를 제공

-> 그래프에서 오류를 식별하고 병목 현상을 찾는 등의 작업에 매우 유용

첫 번째 단계

* 프로그램을 약간 조정해 텐서보드가 읽을 로그 디렉토리에 그래프 정의 및 학습 통계를 기록

- 프로그램을 실행 할 때마다, 다른 로그 디렉토리를 사용해야 함

- 그렇지 않으면 텐서보드가 다른 실행의 통계를 병합하여 시각화를 엉망

- 간단한 해결책은 로그디렉토리 이름에 timestamp를 포함



In [61]:
from datetime import datetime

now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir='tf_logs'
logdir='{}/run-{}/'.format(root_logdir,now)

In [65]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

그런 다음, construction phase 마지막에 다음 코드를 추가

In [66]:
mse_summary=tf.summary.scalar('MSE', mse)
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

첫 번째 줄 : MSE값을 평가하고 _summary_ 라는 텐서보드 호환 바이너리 로그 문자열(TensorBoard-compatible binary log string)에 쓰는 노드를 그래프에 만듦


두 번째 줄 : 로그 디렉토리의 로그 파일에 summary를 쓰는데 사용할 __FileWriter__ 를 만듦
첫 번째 매개변수는 로그 디렉토리의 경로를 나타내고, 두 번째 매개변수는 시각화하려는 그래프

In [67]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

다음으로 학습 중 정기적으로 __mse_summary__ 노드를 평가하기 위해 실행 단계를 업데이트 해야 함

-> __file_writer__ 를 사용해 event file에 기록 할 수 있는 summary정보가 출력

아래는 업데이트 된 코드

In [70]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown


    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    best_theta = theta.eval()

In [71]:
#마지막으로 프로그램 끝에 fileWriter를 닫는다
file_writer.close()

In [72]:
best_theta

array([[ 2.0703337 ],
       [ 0.8637145 ],
       [ 0.12255152],
       [-0.31211877],
       [ 0.38510376],
       [ 0.00434168],
       [-0.0123295 ],
       [-0.83376896],
       [-0.8030471 ]], dtype=float32)

이제 프로그램을 실행해보자!

로그 디렉토리를 만들고, 이 디렉토리에 그래프 정의와 MSE값을 모두 포함하는 envet file 작성

쉘을 열어 작업 디렉토리로 이동 한 다음, `ls -l tf_logs/run*` 입력해 로그 디렉토리의 내용을 나열

# 10. Name Scopes

신경망과 같은 복잡한 모델을 다룰때, 그래프는 수천 개의 노드로 인해 복잡해 보일 수 있다.

이를 피하기 위해 이름 범위(_name scopes_)를 만들어 관련 노드를 그룹화 할 수 있다.

예를 들어 이전의 코드를 수정하여 `loss`라는 이름 범위 내에서 __error__ 와 __mse__ 를 정의 할 수 있다!



In [74]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

In [76]:
with tf.name_scope('loss') as scope:
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error), name="mse")

In [77]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [78]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255152]
 [-0.31211877]
 [ 0.38510376]
 [ 0.00434168]
 [-0.0123295 ]
 [-0.83376896]
 [-0.8030471 ]]


범위 내에 정의 된 각 op의 이름 앞에는 `loss_2/` 가 붙는다 (왜 loss아니라loss_2이지?)

In [80]:
print(error.op.name)

loss_2/sub


In [81]:
print(mse.op.name)

loss_2/mse


# 11. Modularity

두 개의 ReLU(Rectified Linear Unit)의 출력을 더하는 그래프를 만들고 싶다고 가정.

ReLU는 입력의 선형 함수를 계산, 양수이면 결과를 출력, 그렇지 않으면 0을 출력

아래 코드는 작업을 수행하지만, 꽤 반복적

In [82]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z1, 0., name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add(relu1, relu2, name="output")

이러한 반복적인 코드는 유지하기가 어렵고, 오류가 발생하기 쉬움

하지만, 텐서플로우를 사용하면 DRY(Don't Repeat Yourself)상태를 유지 할 수 있다.

아래 코드는 5개의 ReLU 를 만들고 합계를 출력한다

In [83]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [84]:
file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

name scope를 이용하면, 그래프를 훨씬 명확하게 만들 수 있다.

__relu()__ 함수의 모든 내용을 name scope 내로 이동하기만 하면 된다.

In [85]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")    # not shown
        b = tf.Variable(0.0, name="bias")                             # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                      # not shown
        return tf.maximum(z, 0., name="max")                          # not shown

In [86]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

# 12. sharing Variables

그래프의 다양한 컴포넌트 사이에서 변수를 공유하려면, 간단한 함수를 작성하고, 필요로 하는 함수에 파라미터를 전달하면 됨

예를 들어, 모든 `ReLU`의 `threshold`변수를 공유하기 위해 사용되는 `ReLU threshold`를 제어한다고 생각

이럴 땐 우선 변수를 만든 다음 __relu()__ 함수에 전달만 하면 된다.

In [87]:
reset_graph()

def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

임계(threshold)변수를 사용해 모든 ReLU에 대한 임계 값을 제어할 수 있다.

하지만, 많은 공유 파라미터가 있는 경우 파라미터로 계속 전달해야 하는 것은 힘들다.




많은 사람들이 자신의 모델에 있는 모든 변수를 포함하는 python 딕셔너리를 만들고 이를 모든 함수에 전달.

다른 이들은 각 묘듈에 대한 클래스 작성

또 다른 옵션은 아래 코드처럼, 첫 번째 호출 시 공유 파라미터를 relu() 함수의 속성으로 설정

In [88]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):  # 추가
            relu.threshold = tf.Variable(0.0, name="threshold") #추가
        w_shape = int(X.get_shape()[1]), 1                          # not shown in the book
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, relu.threshold, name="max")

In [89]:
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

텐서플로우는 위의 해결책 보다 더 깔끔하고 모듈식의 코드로 이어질 수 있는 또 다른 옵션 제공

아직 존재하지 않는 경우 `get_variable()` 함수를 사용해 공유 파라미터를 작성하거나

이미 존재하는 경우 이를 재사용

생성 또는 재사용은 `variable_scope()`의 특성에 의해 제어

예를들어, 아래 코드에서는 `relu/threshold` 라는 이름의 변수를 만듦

In [90]:
reset_graph()

with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))

이전에 `get_variable()`을 호출해, 변수가 이미 생성된 경우, 위의 코드는 예외를 발생


이 동작은 실수로 변수를 재사용하는 것을 방지

변수를 재사용 하려면, 변수 범위의 재사용 속성을 True로 설정해, 명시적으로 지정

In [91]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

위 코드는 기존 존재하는 

`relu/threshold` 변수를 가져오거나, 

만약 존재하지 않거나 

또는 `get_variable()` 함수를 이용하여 생성되지 않았을 경우 

        -> 예외 발생
        
그 대신에 scope의 `reuse_variables()` 메소드를 호출해 블록 내에서 reuse 속성을 True로 설정

In [92]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

이제 `relu()` 함수가 파라미터로 전달하지 않고도 threshold 변수에 접근 할 수 있는 모든 요소가 있다!!

In [93]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

위 코드는 먼저 `relu()` 함수를 정의하고, `relu/threshold` 변수를 만들고, `relu()`함수를 호출해 5개의 ReLU를 작성

`relu()`함수는 `relu/threshold` 변수를 재사용하고 다른 ReLU노드를 만듦

In [94]:
file_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
file_writer.close()

threshold 변수는 나머지 ReLU코드가 있는 relu() 함수 외부에서 정의해야 함

이를 해결 하기 위해, 아래 코드를 보자!

In [95]:
reset_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
file_writer.close()

첫 번째 호출 시 `relu()`함수 내에 threshold 변수를 만든 다음, 다음 호출에서 다시 사용

-> `relu()` 함수는 name scope 또는 변수 공유에 대해 걱정 할 필요가 없게 됨

`get_variable()` 을 호출하면, threshold변수를 만들거나 재사용

나머지 코드는 `relu()` 를 다섯 번 호출하며 

첫번째 호출에서는` reuse=False로` 설정하고 

다른 호출에서는 `reues=Ture`로 설정

In [96]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]), 1)                        # not shown in the book
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

In [97]:
file_writer = tf.summary.FileWriter("logs/relu9", tf.get_default_graph())
file_writer.close()